In [1]:
import pandas as pd
from pandas.tseries.offsets import BDay
import vnstock as vn  # Assuming vn is a module for stock data
import ta  # Assuming ta is used for technical indicators

# Assuming these variables and functions are defined somewhere in your code
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 160_000_000

def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

def strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI across 30 and OBV rise
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1

    #Sell Signals: RSI across 70 and OBV down 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1

    return df

def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

def simulate_investment(ticker):
    try:
        data = vn.stock_historical_data(ticker, '2021-01-01', '2024-01-02', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}

        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+2 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid initiating trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                current_value = cash + holdings * data['close'].iloc[i]
                portfolio_values.append(current_value)
                continue

            if current_date in buy_signals:
                # Invest all available cash
                shares_to_buy = int(cash // data['close'][i])
                if shares_to_buy > 0:
                    total_cost = shares_to_buy * data['close'][i]
                    cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            if current_date in sell_signals and holdings > 0:
                current_price = data['close'].iloc[i]
                shares_to_sell = holdings
                if shares_to_sell > 0:
                    revenue = shares_to_sell * current_price
                    holdings -= shares_to_sell

                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    if settlement_date in pending_sell_revenue:
                        pending_sell_revenue[settlement_date] += revenue
                    else:
                        pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

def backtest_multiple_companies(companies):
    results = []
    for company in companies:
        result = simulate_investment(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100
            })
    return pd.DataFrame(results)

companies = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

results_df = backtest_multiple_companies(companies)

results_df.to_csv('result/OBV_t2.csv', index=False)


# Print results
print(results_df)

average_rate_of_return = results_df['Rate of Return'].mean()
print("Average Rate of Return for companies:", average_rate_of_return)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9708\928829718.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9708\928829718.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(cash // data['close'][i])
C:\Users\Admin\AppData\Local\Te

   Company  Final Portfolio Value  Total Profit  Rate of Return
0      SSI              160694540        694540        0.434088
1      BCM              209207344      49207344       30.754590
2      VHM              155986830      -4013170       -2.508231
3      VIC              160793696        793696        0.496060
4      VRE              169262950       9262950        5.789344
5      BVH              140647050     -19352950      -12.095594
6      POW              187228696      27228696       17.017935
7      GAS              233859050      73859050       46.161906
8      ACB              252993350      92993350       58.120844
9      BID              303302720     143302720       89.564200
10     CTG              171810920      11810920        7.381825
11     HDB              282643440     122643440       76.652150
12     MBB              237089740      77089740       48.181088
13     SSB              261602160     101602160       63.501350
14     SHB              181554000      2

C:\Users\Admin\AppData\Local\Temp\ipykernel_9708\928829718.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9708\928829718.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(cash // data['close'][i])
C:\Users\Admin\AppData\Local\Te